# データ同化実習(Lorenz63):粒子フィルタ
本データ同化実習では、簡単なモデルを用いて、代表的なデータ同化手法がどのように働いているのかを確認する。
このNotebookでは、Lorenz63modelを対象として、粒子フィルタによるデータ同化を行う。

## Lorenz63model
対象とするLorenz63モデルを以下のように定式化する。σ=10、r=28、b=8/3である。

\begin{align}
\frac{dx}{dt} &= -σ(x -  y) \\
\frac{dy}{dt} &= -y - xz + rx \\
\frac{dz}{dt} &= xy - bz
\end{align}

## [1]ライブラリをインポートする
* numpy:数値計算を効率的に実施するライブラリ
* matplotlib:計算結果の可視化をするライブラリ
* Lorenz63:Lorenz63modelの時間積分と接線形モデルの計算
* GenerateEnsemble:初期のアンサンブルの生成
* ParticleFilter:粒子フィルタの実施
* Visualization:計算結果の可視化およびRMSEの計算

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import Lorenz63
import ParticleFilter
import GenerateEnsemble
import Visualization

## [2]真値、観測値、誤った初期値からの計算を実施する
本実習では双子実験を行う。双子実験とは，同化手法の動作を確認するために行う試行実験の一つである。ある初期値を仮に「真の初期値」とみなして数値実験を行い、その数値実験結果に誤差を加えたものを「観測値」と呼ぶ。そして，「誤った初期値」から始めた数値実験結果に「観測値」を同化することによって、「真の初期値」や「真の数値実験結果」が復元できれば、同化手法が（双子実験の枠組みの中でではあるが）有効であることがわかる。

そこで、このNotebookでは、最初に真の初期値から計算した真値(True)、そこに誤差を加えた観測値(Obs.)、誤った初期値から計算した(Sim.)の3つについて、データ同化を実施する前に計算を行うこととする。

#### 設定すべきパラメータについて
* nt_asm:データ同化を実施する時間ステップ数
* nt_prd:データ同化を実施しない予測ステップ数
* noise:観測値に与える誤差の標準偏差
* xt, yt, zt: 真の初期値
* xsf, ysf, zsf: 誤った初期値

#### 設定しても良いパラメータについて
以下のパラメータは設定しなくても適切に計算されるように初期設定を与えている。もし変更したい場合は、__Lorenz63__関数を呼び出しているところで、引数に追加することで変更可能。
* dt:時間積分のタイムステップ(default=0.01)
* s:σ(default=10.0)
* r:r(default=28.0)
* b:b(default=8/3)

In [ ]:
#time step
nt_asm = 1000
nt_prd = 1000

#Observation noise
noise = 0.1 #standard deviation

#Initial condition
xt = 10.0; yt = 14.0; zt = 24.0 # #True
xsf = 11.0; ysf = 13.0; zsf = 25.0  #Simulation

#For Visuallization
xtrue = [xt];ytrue = [yt]; ztrue = [zt] #True for visualization
xobs = [xt];yobs = [yt];zobs = [zt] #Observation for visualization
xsim = [xsf]; ysim = [ysf]; zsim = [zsf] #Simulation for visualization
xs = xsf; ys = ysf; zs = zsf

for t in range(nt_asm + nt_prd):
    #True,Observation and Simulation
    lorenz = Lorenz63.Lorenz63(xt,yt,zt,noise)
    xt, yt, zt = lorenz.time_integration()
    xo, yo, zo = lorenz.observation_noise()
    
    #Simulation
    lorenz = Lorenz63.Lorenz63(xs,ys,zs,noise)
    xs, ys, zs = lorenz.time_integration()
    
    #visualization
    xtrue.append(xt);ytrue.append(yt);ztrue.append(zt)
    xobs.append(xo);yobs.append(yo);zobs.append(zo)
    xsim.append(xs);ysim.append(ys);zsim.append(zs)

xtrue = np.array(xtrue)
xsim = np.array(xsim)
xobs = np.array(xobs)

In [ ]:
fig, ax = plt.subplots(figsize=(10,3))
it = np.arange(nt_asm + nt_prd + 1)
t = it * lorenz.dt
mask_obs = (it % 100 == 0) & (it < nt_asm) & (it > 0)
plt.plot(t, xtrue, '-', c='C0', label='True')
plt.plot(t, xsim, '-', c='C1', label='Simulation')
plt.plot(t[mask_obs], xobs[mask_obs], '^', c='C2', label='Observation')
plt.xlim(t.min(), t.max())
plt.legend()
plt.show()

## [3]データ同化を実施する

### 1) 粒子フィルタ

#### 設定すべきパラメータについて
* obs_interval:データ同化の間隔ステップ数
* nens:粒子の数
* H: 観測行列

In [ ]:
obs_interval = 100
it = np.arange(nt_asm + nt_prd + 1, dtype=np.int64)
mask_obs = (it > 0) & (it < nt_asm) & (it % obs_interval == 0)

# 観測行列
H = np.array([1,0,0]).reshape(1,3)

xda = [xsf];yda = [ysf];zda = [zsf] # Data Assimilation for visualization
xs = xsf; ys = ysf; zs = zsf        # Simulation
xyzsim = np.array([xs, ys, zs])

nens = 1000
initial_noise = 4.0
generate =  GenerateEnsemble.GenerateEnsemble(xyzsim, nens, initial_noise)                                      
ensemble = generate.fit()

In [ ]:
# 粒子フィルタの動作を確認するため同化前後における粒子（Xの値のみ）を保存する。
x_prior = []
x_posterior = []

In [ ]:
for t in range(nt_asm + nt_prd):
    # Data Assimilation
    for ens in range(nens):
        lorenz = Lorenz63.Lorenz63(ensemble[ens,0], ensemble[ens,1], ensemble[ens,2], noise, None)
        ensemble[ens,0], ensemble[ens,1], ensemble[ens,2] = lorenz.time_integration()

    if mask_obs[t+1]:
        xyzobs = np.array([xobs[t+1],yobs[t+1],zobs[t+1]])
        pf = ParticleFilter.ParticleFilter(ensemble, H, xyzobs, 0.5)
        update = pf.fit()
        x_prior.append(ensemble[:,0].copy())
        x_posterior.append(update[:,0].copy())
        ensemble = update

    xda.append(ensemble[:,0].mean())
    yda.append(ensemble[:,1].mean())
    zda.append(ensemble[:,2].mean())

xda = np.array(xda)
yda = np.array(yda)
zda = np.array(zda)

In [ ]:
fig, ax = plt.subplots(figsize=(10,3))
t = it * lorenz.dt
plt.plot(t, xtrue, '-', c='C0', label='True')
plt.plot(t, xsim, '-', c='C1', label='Simulation')
plt.plot(t[mask_obs], xobs[mask_obs], '^', c='C2', label='Observation')
plt.plot(t, xda, '-', c='C3', label='DA')
plt.xlim(t.min(), t.max())
plt.legend()
plt.show()

In [ ]:
obs = xobs[mask_obs]

fig, axs = plt.subplots(nrows=2, figsize=(6,8), sharex=True, sharey=True)

i = 0
axs[0].hist(x_prior[i], density=True, fc='#AEC7E8', label='Prior')
axs[1].hist(x_posterior[i], density=True, fc='#FFBB78', label='Posterior')
axs[0].axvline(x=obs[i], ls='--', c='C3', lw=2.0, label='Observation')
axs[1].axvline(x=obs[i], ls='--', c='C3', lw=2.0, label='Observation')
axs[1].set_xlabel(r'$x$')

for ax in axs:
    ax.set_ylabel('probability density')
    ax.legend(loc='upper right')
    ax.grid(ls='--', c='k', lw=0.5, dashes=(3,2))

plt.tight_layout()
plt.show()

### 2) 融合粒子フィルタ

#### 概要

#### 設定すべきパラメータについて
* obs_interval:データ同化の間隔ステップ数
* nens:粒子の数
* H: 観測行列

In [ ]:
obs_interval = 100
mask_obs = (it > 0) & (it < nt_asm) & (it % obs_interval == 0)

H = np.array([1,0,0]).reshape(1,3)

xda = [xsf];yda = [ysf];zda = [zsf] # Data Assimilation for visualization
xs = xsf; ys = ysf; zs = zsf        # Simulation
xyzsim = np.array([xs, ys, zs])

nens = 1000
initial_noise = 4.0
generate =  GenerateEnsemble.GenerateEnsemble(xyzsim, nens, initial_noise)                                      
ensemble = generate.fit()

In [ ]:
x_prior = []
x_posterior = []

In [ ]:
for t in range(nt_asm + nt_prd):
    # Data Assimilation
    for ens in range(nens):
        lorenz = Lorenz63.Lorenz63(ensemble[ens,0], ensemble[ens,1], ensemble[ens,2], noise, None)
        ensemble[ens,0], ensemble[ens,1], ensemble[ens,2] = lorenz.time_integration()

    if mask_obs[t+1]:
        xyzobs = np.array([xobs[t+1],yobs[t+1],zobs[t+1]])
        pf = ParticleFilter.MergingParticleFilter(ensemble, H, xyzobs, 0.5)
        update = pf.fit()
        x_prior.append(ensemble[:,0].copy())
        x_posterior.append(update[:,0].copy())
        ensemble = update

    xda.append(ensemble[:,0].mean())
    yda.append(ensemble[:,1].mean())
    zda.append(ensemble[:,2].mean())

xda = np.array(xda)
yda = np.array(yda)
zda = np.array(zda)

In [ ]:
fig, ax = plt.subplots(figsize=(10,3))
it = np.arange(nt_asm + nt_prd + 1)
t = it * lorenz.dt
mask_obs = (it % 100 == 0) & (it < nt_asm) & (it > 0)
plt.plot(t, xtrue, '-', c='C0', label='True')
plt.plot(t, xsim, '-', c='C1', label='Simulation')
plt.plot(t[mask_obs], xobs[mask_obs], '^', c='C2', label='Observation')
plt.plot(t, xda, '-', c='C3', label='DA')
plt.xlim(t.min(), t.max())
plt.legend()
plt.show()

In [ ]:
obs = xobs[mask_obs]

fig, axs = plt.subplots(nrows=2, figsize=(6,8), sharex=True, sharey=True)

i = 0
axs[0].hist(x_prior[i], density=True, fc='#AEC7E8', label='Prior')
axs[1].hist(x_posterior[i], density=True, fc='#FFBB78', label='Posterior')
axs[0].axvline(x=obs[i], ls='--', c='C3', lw=2.0, label='Observation')
axs[1].axvline(x=obs[i], ls='--', c='C3', lw=2.0, label='Observation')
axs[1].set_xlabel(r'$x$')

for ax in axs:
    ax.set_ylabel('probability density')
    ax.legend(loc='upper right')
    ax.grid(ls='--', c='k', lw=0.5, dashes=(3,2))

plt.tight_layout()
plt.show()